In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime,timedelta
import pandas as pd
import re

In [84]:
host = 'https://www.moneycontrol.com'
base_url = 'https://www.moneycontrol.com/stocks/marketstats/industry-classification/bse/abrasives.html'
about_us_data = []

data_struct = {
    'Name':"",
    "Sector":"",
    "Link":"",
    "Website":"",
    "About us":""
}

In [3]:
response = requests.get(base_url)
soup = BeautifulSoup(response.text,'html.parser')  

In [75]:
industries = []

In [79]:
for index,li in enumerate(soup.find("ul",{"class":"IndustryWeb_stat_lflist__Colzh"}).children):
        industry_data = {}
        industry_data['name'] = li.text
        industry_data['link'] = f"{host}{li.find('a')['href']}"
        industries.append(industry_data)

In [80]:
industries

[{'name': 'Abrasives',
  'link': 'https://www.moneycontrol.com/stocks/marketstats/industry-classification/bse/abrasives.html'},
 {'name': 'Aerospace & Defence',
  'link': 'https://www.moneycontrol.com/stocks/marketstats/industry-classification/bse/aerospace-defence.html'},
 {'name': 'Agriculture',
  'link': 'https://www.moneycontrol.com/stocks/marketstats/industry-classification/bse/agriculture.html'},
 {'name': 'Air Conditioners',
  'link': 'https://www.moneycontrol.com/stocks/marketstats/industry-classification/bse/air-conditioners.html'},
 {'name': 'Airlines',
  'link': 'https://www.moneycontrol.com/stocks/marketstats/industry-classification/bse/airlines.html'},
 {'name': 'Aluminium & Aluminium Products',
  'link': 'https://www.moneycontrol.com/stocks/marketstats/industry-classification/bse/aluminium-aluminium-products.html'},
 {'name': 'Amusement Parks/Recreation/Club',
  'link': 'https://www.moneycontrol.com/stocks/marketstats/industry-classification/bse/amusement-parksrecreationc

In [87]:
for i in industries[:10]:
    companies_data = get_companies_in_sector(i['link'],i['name'])
    about_us_data.append(companies_data)

ConnectionError: HTTPSConnectionPool(host='www.moneycontrol.com', port=443): Max retries exceeded with url: /stocks/marketstats/industry-classification/bse/abrasives.html (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001BB64244400>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [65]:
def get_company_info(link,d=data_struct.copy()):
    response = requests.get(link)
    comp_soup =  BeautifulSoup(response.text,'html.parser')
    name = comp_soup.find('div',{"id":"stockName"}).find("h1").text
    about_us  = comp_soup.find('div',{"id":"company_info"})
    info_div = about_us.find("div",{"class":"morepls_cnt"})
    if info_div:
        d['About us'] = info_div.text
    comp_links = about_us.find("ul",{'class':"comdetl"}).find_all('a')
    if len(comp_links) > 0:
        d['Website'] = comp_links[-1]['href']
        
    d['Name']=name
    d['Link'] = link
    
    return d

In [86]:
def get_companies_in_sector(industry_link,industry_name=""):
    i_data = []
    response = requests.get(industry_link)
    print(f"------------------------{industry_name}-------------------")
    comp_in_sectors_soup =  BeautifulSoup(response.text,'html.parser')  
    for com in comp_in_sectors_soup.find('div',{"class":"ReuseTable_bsr_table__eVZZf"}).find('tbody').children:
        d = data_struct.copy()
        anker = com.find('td').find('a')
        print(anker.text)
        com_link = anker['href']
        d['Sector'] = industry_name
        comp_data = get_company_info(com_link,d)
        i_data.append(comp_data)
    return i_data

In [72]:
get_companies_in_sector(industry_link,"Abrasives")